In [1]:
import os.path as osp

import mmcv
from mmengine.fileio import dump, load
from mmengine.utils import track_iter_progress


import glob 


In [2]:
def convert_to_coco(image_prefix,out_file):
    images = []
    annotations = []

    obj_count = 0
    for idx,image_pathname in enumerate(track_iter_progress(glob.glob(osp.join(image_prefix,'images','*')))):
        image_name = image_pathname.split('/')[-1]
        label_pathname =  glob.glob(osp.join(image_prefix,'labels',f'{image_name[:-4]}*'))[0]
        
        height, width = mmcv.imread(image_pathname).shape[:2]
        images.append(
            dict(id=idx, file_name=image_name, height=height, width=width))
        
        
        with open(label_pathname) as file :

            data_infos = file.readlines()
            for data_info in data_infos:
                category_id,x_center, y_center, w_bbox, h_bbox = [float(i) for i in data_info.split(' ')]
                
                x_center = int(x_center * width)
                y_center = int(y_center * height)
                w_bbox   = int(w_bbox * width)
                h_bbox   = int(h_bbox * height)
                
                x_min = int(x_center - w_bbox / 2)
                y_min = int(y_center - h_bbox / 2)


                annotations.append(dict(
                            image_id=idx,
                            id=obj_count,
                            category_id=int(category_id),
                            bbox=[x_min, y_min, w_bbox, h_bbox],
                            area=w_bbox * h_bbox,
                            segmentation=[],
                            iscrowd=0))
                obj_count += 1
    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{
            'id': 0,
            'name': 'round'
        },
        {
            'id': 1,
            'name': 'shape'
        }])
    dump(coco_format_json, out_file)

In [3]:

train_pathname = '/home/neau/sdb/datasets/leafs/train'
val_pathname = '/home/neau/sdb/datasets/leafs/val'

convert_to_coco(train_pathname,'/home/neau/sdb/datasets/leafs/train/annotation_coco.json')
convert_to_coco(val_pathname,'/home/neau/sdb/datasets/leafs/val/annotation_coco.json')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 116/116, 27.8 task/s, elapsed: 4s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 28/28, 30.0 task/s, elapsed: 1s, ETA:     0s
